# 목표

- 데이터를 보고 HS부호를 뽑는 프로세스를 이해하기
- 추천 시스템과 RAG 중 어떤 방법을 선택할지 정하기
- 추가할 데이터는 없는지 고려해보기

# Library

In [26]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

# Data Load

In [27]:
text_data = pd.read_csv('./data/비식별된 해외기업별 영문 텍스트데이터.csv', dtype=str)
statistics_data = pd.read_csv('./data/통계청 국제표준산업분류 HSCODE 6단위 매핑.csv', dtype=str)
customs_data = pd.read_csv('./data/관세청_HS부호_240101.csv', dtype=str)

In [28]:
text_data_copy = text_data.copy()
statistics_data_copy = statistics_data.copy()
customs_data_copy = customs_data.copy()

In [29]:
display(text_data_copy.info())
display(statistics_data_copy.info())
display(customs_data_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10000 non-null  object
 1   CODE    10000 non-null  object
 2   DSC     10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8519 entries, 0 to 8518
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ISIC4_국제표준산업분류     2970 non-null   object
 1   ISIC4_분류명          2963 non-null   object
 2   KSIC10_한국표준산업분류    3732 non-null   object
 3   KSIC10_분류명         3729 non-null   object
 4   HS2017_관세통계통합품목분류  6061 non-null   object
 5   HS2017_분류명         6061 non-null   object
dtypes: object(6)
memory usage: 399.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12422 entries, 0 to 12421
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호       12422 non-null  object
 1   한글품목명      12422 non-null  object
 2   영문품목명      12422 non-null  object
 3   성질통합분류코드   11294 non-null  object
 4   성질통합분류코드명  11294 non-null  object
dtypes: object(5)
memory usage: 485.4+ KB


None

# 중복값 확인

## 텍스트

In [30]:
text_data_copy[text_data_copy.duplicated(keep=False)]

,ID,CODE,DSC
4393,4555,4659,"commercial equipment, nec the wholesale distr..."
4394,4555,4659,"commercial equipment, nec the wholesale distr..."
9911,10563,2013,"plastics materials or resins, nec manufacturi..."
9912,10563,2013,"plastics materials or resins, nec manufacturi..."
9930,10579,2100,"pharmaceutical preparations manufacturing, fa..."
9931,10579,2100,"pharmaceutical preparations manufacturing, fa..."


In [31]:
print(text_data_copy.shape)
print(text_data_copy[text_data_copy.duplicated()].shape)
print(text_data_copy.drop_duplicates().shape)

(10000, 3)
(3, 3)
(9997, 3)


In [32]:
text_data_copy = text_data_copy.drop_duplicates()
text_data_copy = text_data_copy.reset_index(drop=True)

## 통계청

In [33]:
statistics_data_copy[statistics_data_copy.duplicated(keep=False)]

,ISIC4_국제표준산업분류,ISIC4_분류명,KSIC10_한국표준산업분류,KSIC10_분류명,HS2017_관세통계통합품목분류,HS2017_분류명
10,0111,"곡물(쌀 제외), 콩류, 종실유 재배업",01123,종자 및 묘목 생산업,100310,종자
11,0111,"곡물(쌀 제외), 콩류, 종실유 재배업",01110,곡물 및 기타 식량작물 재배업,100390,기타
12,0111,"곡물(쌀 제외), 콩류, 종실유 재배업",01123,종자 및 묘목 생산업,100310,종자
13,0111,"곡물(쌀 제외), 콩류, 종실유 재배업",01110,곡물 및 기타 식량작물 재배업,100390,기타
21,NaN,NaN,01123,종자 및 묘목 생산업,NaN,NaN
...,...,...,...,...,...,...
8493,NaN,NaN,96912,가정용 세탁업,NaN,NaN
8495,9601,직물 및 모피 제품 세탁과 (드라이) 클리닝,96911,산업용 세탁업,NaN,NaN
8496,NaN,NaN,96912,가정용 세탁업,NaN,NaN
8497,9601,직물 및 모피 제품 세탁과 (드라이) 클리닝,96911,산업용 세탁업,NaN,NaN


In [34]:
print(statistics_data_copy.shape)
print(statistics_data_copy[statistics_data_copy.duplicated()].shape)
print(statistics_data_copy.drop_duplicates().shape)

(8519, 6)
(1671, 6)
(6848, 6)


In [35]:
statistics_data_copy = statistics_data_copy.drop_duplicates()
statistics_data_copy = statistics_data_copy.reset_index(drop=True)

## 관세청

In [36]:
customs_data_copy[customs_data_copy.duplicated()]

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명


# 결측치 채우기 (보류)

- 윗줄꺼로 보간은 아닌 것 같음. 결측치를 채우기 위한 교차 확인이 되지 않음. 일단.. 결측치 있는채로 진행

In [37]:
# 일단 공백으로 채우기
text_data_copy = text_data_copy.fillna('')
statistics_data_copy = statistics_data_copy.fillna('')
customs_data_copy = customs_data_copy.fillna('')

# 텍스트 -> 통계청, 관세청

In [38]:
# 영문 텍스트 확인
text_data_copy.iloc[0]

ID                                                      1
CODE                                                 4520
DSC     automotive repair shops, nec  specialized auto...
Name: 0, dtype: object

In [39]:
# 영문 텍스트 확인
text_data_copy.iloc[0, 2]

'automotive repair shops, nec  specialized automotive repair, not elsewhere classified, such as fuel service carburetor repair, brake relining, front-end and wheel alignment, and radiator repair. motor vehicle repair and maintenance auto brake lining, installation other automotive mechanical and electrical repair and maintenance maintenance and repair of motor vehicles maintenance and repair of motor vehicles maintenance and repair of motor vehiclesother automotive repair and maintenance'

자동차 수리점, 기타 특별한 자동차 수리, 연료 서비스, 카뷰레터 수리, 브레이크 라이닝 교체, 

전면 및 휠 정렬, 라디에이터 수리 등. 자동차 수리 및 유지보수, 자동차 브레이크 라이닝 설치, 

기타 자동차 기계 및 전기 수리 및 유지보수, 자동차의 유지보수 및 수리, 자동차의 유지보수 및 수리, 기타 자동차 수리 및 유지보수.

In [40]:
# 텍스트 데이터의 CODE를 통계청에서 확인
statistics_data_copy[statistics_data_copy['ISIC4_국제표준산업분류'] == '4520']

,ISIC4_국제표준산업분류,ISIC4_분류명,KSIC10_한국표준산업분류,KSIC10_분류명,HS2017_관세통계통합품목분류,HS2017_분류명
7630,4520,자동차 정비 및 수리업,95211,자동차 종합 수리업,,
7631,4520,자동차 정비 및 수리업,95212,자동차 전문 수리업,,
7632,4520,자동차 정비 및 수리업,95213,자동차 세차업,,


- 영문 텍스트를 해석하면 7630과 7631행과 관련이 있는 것으로 보임
- 관세청에서 자동차 수리 관련 행을 찾아야 할 듯

In [41]:
customs_data_copy.head()

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
0,0101211000,농가 사육용,For farm breeding,11020101,말
1,0101219000,기타,Other,11020101,말
2,0101291000,경주말,Horses for racing,11020101,말
3,0101299000,기타,Other,11020101,말
4,0101300000,당나귀,Asses,11020190,기타 산 동물


In [52]:
idx_list_0 = []
idx_list_1 = []
for idx in range(0, customs_data_copy.shape[0]):
    # 먼저 한글품목명 기준으로 키워드를 찾는다.
    cond = customs_data_copy.iloc[idx, 1]
    if '자동차' in cond: # 수리, 자동차, 부품
        # print(idx, end=' ')
        idx_list_0.append(idx)
        # 한글품목명으로 걸러진 데이터에서 성질통합분류코드명 기준으로 키워드를 찾는다.
        cond2 = customs_data_copy.iloc[idx, 4]
        if '부품' in cond2:
            # print(idx, end=' ')
            idx_list_1.append(idx)

In [54]:
customs_data_copy.iloc[idx_list_1] # idx_list_0

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
8332,7320101000,자동차용 판상 스프링,Leaf-springs for automobiles,31060101,자동차 부품
8335,7320201000,자동차용,For automobiles,31060101,자동차 부품
10835,8536902000,"제8702호, 제8703호, 제8704호나 제8711호의 자동차용 배터리 클램프(b...",Battery clamp of a kind used for motor vehicle...,31060101,자동차 부품
10849,8538902000,자동차단기의 것,Of automatic circuit breakers,32040505,전기공급 및 제어장치 부품


In [57]:
idx_list_2 = []
for idx in range(0, customs_data_copy.shape[0]):
    # 영문품목명 기준으로 키워드를 찾는다.
    cond = customs_data_copy.iloc[idx, 2]
    if 'auto' in cond: # auto
        # print(idx, end=' ')
        idx_list_2.append(idx)

In [58]:
customs_data_copy.iloc[idx_list_2]

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
2630,2710197120,자동차용 기관오일,Engine oil for automotive,23020206,윤활유
2639,0027101973,"콤파운드오일, 유동파라핀과 자동변속기용 윤활유","Compound oil, liquid paraffin and automatic tr...",,
2691,2710209712,자동차용 기관오일,Engine oil for automotive,23020206,윤활유
2700,0271020973,"콤파운드오일, 유동파라핀과 자동변속기용 윤활유","Compound oil, liquid paraffin and automatic tr...",,
8332,7320101000,자동차용 판상 스프링,Leaf-springs for automobiles,31060101,자동차 부품
...,...,...,...,...,...
12032,9302009021,반자동,Semi-automatic,32010401,무기 및 총포탄
12040,9303201012,반자동,Semi-automatic,12900101,기타 내구소비재
12047,9303209012,반자동,Semi-automatic,12900101,기타 내구소비재
12053,9303301020,반자동,Semi-automatic,12900101,기타 내구소비재


- 데이터를 보고 HS부호를 뽑는 프로세스를 이해하기 
    - 완료
    - 근데 관세청에서 한글과 영어 중 어떤 것이 더 좋은지 모르겠음.. 한글과 영어를 혼합해서 찾아야 할 것 같음
- 추천 시스템과 RAG 중 어떤 방법을 선택할지 정하기
    - RAG가 더 좋을 것 같음..!
- 추가할 데이터는 없는지 고려해보기
    - 포털에서 통계청, 관세청의 HS부호로 영문으로 되어있는 문장을 추가로 넣어줄 수 있을 것 같음
    - 영문 텍스트 데이터와 연관성이 더 높아질 것 같음
    - 추가 데이터로 관세청에서 HS부호를 더 적절히 추출할 수 있을 것 같음